# Query/Context Dataset Generation
***

This notebook walks students through the process of generating datasets of query/context pairs which can be used for two primary purposes:
- Fine-tuning an embedding model
- Serve as ground truth for retrieval evaluation

In [1]:
from retrieval_evaluation import QueryContextGenerator
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from prompt_templates import qa_generation_prompt
from preprocessing import FileIO
from rich import print
import pandas as pd
import os

from dotenv import load_dotenv
env = load_dotenv('.env', override=True)

/Users/davide/Documents/MooC/Corise/VectorSearch/vectorsearch-applications/impactenv/lib/python3.11/site-packages/llama_index/download/download_utils.py:11: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/Users/davide/Documents/MooC/Corise/VectorSearch/vectorsearch-applications/impactenv/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)


In [2]:
#instantiate generate with openai_key, model_id default is 'gpt-3.5-turbo-0613'
generator = QueryContextGenerator(openai_key=os.environ['OPENAI_API_KEY'])

### Load raw data
Load raw data from parquet file.  Raw data should be in the same format as the dataset (corpus) created in [Notebook 1](https://github.com/americanthinker/vectorsearch-applications/blob/main/1-Data_Preprocessing_Week1_COLAB.ipynb). 

In [6]:
data_path = './impact-thoery-minilmL6-256.parquet'
data = FileIO().load_parquet(data_path)
len(data)

Shape of data: (26448, 12)
Memory Usage: 2.42+ MB


26448

### Data Length Analysis
Conduct an analysis of the length of the content chunks.  Can use either raw words or tokens to assess length.  The main point here is to get a sense of the mean length of content chunks in the data and to set the `total_chars` param in the `clean_validate_data` method with an appropriate value.

In [8]:
#in this example the mean content length is @ 1,000
lengths = [len(d['content']) for d in data]
df = pd.DataFrame(lengths)
df.describe() 

,0
count,26448.000000
mean,991.729053
std,126.344870
min,4.000000
25%,944.000000
50%,1005.000000
75%,1060.000000
max,1974.000000


### Split Data

The `train_val_split` function will clean and validate the raw data as a first step and then split into user defined train/val splits.  
- Cleaning simply strips the keys from the data that are not needed for the query/content generation process
- Validation consists of ensuring that only content chunks of length > `total_chars` are passed to the LLM (this step prevents the LLM from asking questions from sparse context)

Users define the number of training samples and validation samples to generate.  Number of questions per content chunk can also be set to more than 1, however a note of caution:
- Setting `num_questions_per_chunk` > 1 saves time (and money) by asking more than one question per content chunk, however, the dataset will be less diverse.  There is also the potential for the model to generate lower quality questions if the content chunk isn't large enough or meaningful enough to generate more than one question from the content.
- Retrieval evaluation results from fine-tuning an embedding model with 200-300 training samples showed an uptick of 5-10% points.  Upper bound on retrieval improvement as a funtion of training sample size is yet to be determined (have fun pushing the boundaries! 👊)
- A validation data set is not required for seeing improvement from fine tuning.  The addition of a validation dataset, however, allows a user to test the results of fine tuning on an unseen dataset. 

In [9]:
#split data into train/val sets
#in this example we are creating a training set of n=10, val set of n=5, and asking the LLM to only ask 1 question per chunk. 
train, val = generator.train_val_split(data, 10, 5, 1, total_chars=950)

Length Training Data: 10
Length Validation Data: 5


### Generate QA pairs

To generate query/context pairs we need to pass in our cleaned data splits, a question asking generation prompt, and the number of questions per chunk (needs to be same value passed into the `train_val_split` function.
The `qa_generation_prompt` is already preconfigured and supplies the LLM with additional context about the Impact Theory show to ensure high quality questions are asked given the additional context.   
Print out the prompt to see what is being asked of the model:

In [12]:
print(qa_generation_prompt)

Impact Theory episode summary and episode guest are below:

---------------------
Summary: {summary}
---------------------
Guest: {guest}
---------------------
Given the Summary and Guest of the episode as context use the following randomly selected transcript section \  
of the episode and not prior knowledge, generate questions that can be answered by the transcript section: 

---------------------
Transcript: {transcript}
---------------------

Your task is to create {num_questions_per_chunk} questions that can only be answered given the previous context and
transcript details. The question should randomly start with How, Why, or What.

The output from this function is a llama_index class `EmbeddingQAFinetuneDataset`, which is a simple wrapper for a series of three dictionaries (`corpus`, `queries`, and `relevant_docs`).  The llama_index class is not absolutely necessary, but it is helpful in making transitions smoother when using the llama_index `SentenceTransformersFinetuneEngine` class for fine-tuning.  It takes roughly 80 seconds to generate 100 query/context pairs so a sample size of 300 takes about 4 minutes (much faster than if you were to do this manually!).

In [14]:
training_set = generator.generate_qa_embedding_pairs(train, qa_generation_prompt, 1)
val_set = generator.generate_qa_embedding_pairs(val, qa_generation_prompt, 1)

100%|██████████| 5/5 [00:06<00:00,  1.38s/it]


In [15]:
#EmbeddingQAFinetuneDataset has no len, so check length of queries instead
len(training_set.queries), len(val_set.queries)

(10, 5)

### Dataset Analysis

Always a good idea to check the quality of the pairs generated.  Most pairs will be high quality but some will not be, this is a chance for human intervention to adjust the questions manually to ensure the quality remains high. 

In [16]:
def show_qa_pairs(data: EmbeddingQAFinetuneDataset, print_results: bool=True):
    pairs = []
    for k, v in data.queries.items():
        doc_id = data.relevant_docs[k][0]
        context = data.corpus[doc_id]
        pairs.append((v, context))
    if print_results:
        for tup in pairs:
            print(f'Question: {tup[0]}\nContext: {tup[1]}\n\n')
    return pairs    

In [17]:
pairs = show_qa_pairs(val_set, print_results=True)

Question: How did Colin O'Brady's tears react when he cried in the extreme cold?
Context: And I realized, to me, I say the only question that truly matters when facing those kinds of obstacles is 
how will we respond? I'm literally crying at this moment. When it's minus 25, what happens when you cry? Your tears
freeze to your face. It's a pathetic sight, like all around. But I said to myself, and actually with the guidance 
of Jenna, who's certainly been, you know, a huge guiding force to me, if I haven't mentioned her enough already, 
I'll continue to. And she goes, look, how far away are you from the first waypoint? And, you know, on my GPS, I've 
got these, like, markings on the map. And I, at the time, I'm like, you know, I'm 0.54 miles away from the first. 
There's millions of miles. She's like, okay, so you're half a mile away from the first waypoint. Forget about the 
thousand mile journey. Forget about Captain Lou that's, like, already beating you in this race. Forget about the 
media, the press, the students falling all in the classroom. Forget about that.

Question: How does Jay Samit encourage people to overcome the classic excuse of running out of money in business?
Context: You're saying when people, the classic excuse is, oh, I ran out of money. Business was right there, but I 
ran out of money. The same thing is I've seen it countless times in larger companies where people are like, I have 
this brilliant idea, but they don't get it like you were saying. When you take the ownership yourself and you put 
yourself in the driver's seat, I think you get a step closer to what I think really has made you so great and so 
successful. It's certainly what turned me on so much about your book, which is basically Ryan Holiday's concept of 
the obstacle is the way. It's really amazing when you come up with a big, hairy, audacious idea, goal, something. 
You want to get me motivated. There's only one thing you have to say to me. You can't. That's it. Tom Bilyeu. Thank
you to you. I'm like ... It was funny, my editor of my book said, you're very competitive because I never played 
sports. I'm like, I'm not competitive. I just have to win. I never thought of myself as that.

Question: How does Sam Harris define everyday truth and what factors does he believe influence it?
Context: Like even that I think is a task, but let's say that we all agree that we're only going to say what's 
true. Now this gets really complicated. And I will put forth that what I think is quote unquote true is based on 
perspective, interpretation and reinforcement. So there's physics, which we don't even understand fully. And then 
there's everything else. And because a gigantic part of everyday truth, and I don't know if that will help us get 
to a sort of in the weeds working definition, but everyday truth seems to be predicated on that. You have to take 
into account the person's perspective. So how do they see the world? Blue, red, right? You have to take into 
account their interpretation. So looking at data, some people are going to say, no, it doesn't show that. It shows 
this. And you can get people that look at the data and just violently disagree on what it shows. And then you've 
got the reinforcement. So if they put out a tweet saying their version of the truth and they get a wall of 
reinforcement, then they're just one that feels good.

Question: How does David Goggins describe the concept of the governor and its impact on human capability?
Context: I had to armor plate my mind and It's about what you're saying to yourself, but it also comes at work So 
whenever I was getting beat down physically mentally spiritually whenever I was going through just saying, you know
I would put you know, you can't hurt me can't hurt me just became a message. I you know, I would say to myself and 
That's just kind of where it comes from What I find so interesting is your concept of the governor that basically 
I've the chills that an expert is somebody who's gonna tell you What your limits are rather than the person that's 
out there practicing getting you beyond the limit So what is the governor and how do we strip it out of our lives? 
I believe that most human beings are only living at about 40% of their capability so The mind has a governor like a
car if you're driving a car and the car has a governor on it The car may say 130 miles an hour, but the governor 
set for 91 Once that governor sets in you get to 91 that car starts doing this the car wants to go The car wants to
go but that fucking factory said, uh-uh.

Question: How does Chip Conley describe curiosity as an elixir for life and a key to fostering creativity?
Context: I had to move from this place of trying to win, and my definition of success was chips wins to being in a 
place of like, I think I'm learning. And I was. And if you can be learning in your mid-50s a bunch of things you 
didn't know before, you're living. And that curiosity becomes sort of an elixir for life. It's like a 
life-affirming spirit inside of you. How do you get people to do, to foster their creativity, to push it? I'm sure 
people come up to you and ask like, okay, if creativity is that core thing that's going to make my wrinkles 
disappear, how do I get more of it? Well, it's curiosity. Curiosity is the elixir for creativity. So creativity and
innovation is what we tend to focus on. But I think behind creativity and innovation is this idea of curiosity. 
What's behind curiosity is a lack of fear to ask naive questions. A four-year-old doesn't edit themselves when they
say, why is the sky blue? Four-year-olds ask a lot of questions.

### Save to Disk  
Save to disk using your own filepaths, below is an example using the length of the sets as part of the filepath.

In [18]:
training_set.save_json('./data/training_data_10.json')
val_set.save_json('./data/validation_data_5.json')